Setup and Data Loading

In [ ]:

import os
import glob
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

results_dir = '../results'
plots_dir = '../plots'
os.makedirs(plots_dir, exist_ok=True)

# Load CSVs
score_files = glob.glob(os.path.join(results_dir, "*_scores.csv"))
scores = {}
for file in score_files:
    name = os.path.basename(file).replace("_scores.csv", "")
    df = pd.read_csv(file)
    scores[name] = df['score']
print(scores)

{}


In [7]:
import os
print("Current working dir:", os.getcwd())

Current working dir: c:\Users\jlawi\OneDrive - Georgia Institute of Technology\Documents\Simulation Project\analysis


Summary Statistics

In [ ]:
summary = []
for name, data in scores.items():
    mean = round(data.mean(), 2)
    median = data.median()
    std = round(data.std(), 2)
    ci = round(1.96 * std / math.sqrt(len(data)), 2)
    summary.append({
        "Strategy": name,
        "Mean": mean,
        "Median": median,
        "Std Dev": std,
        "Min": data.min(),
        "Max": data.max(),
        "95% CI": ci
    })
summary_df = pd.DataFrame(summary).sort_values("Mean", ascending=False)
summary_df

RangeIndex(start=0, stop=0, step=1)


KeyError: 'Mean'

Histogram and Boxplots

In [ ]:
for name, data in scores.items():
    plt.figure()
    plt.hist(data, bins=30)
    plt.title(f"{name} – Score Distribution")
    plt.xlabel("Total Score")
    plt.ylabel("Frequency")
    plt.show()

plt.figure()
plt.boxplot(scores.values(), labels=scores.keys())
plt.title("Strategy Comparison – Boxplot")
plt.ylabel("Total Score")
plt.show()

Pairwise t-Tests

In [ ]:
print("Pairwise t-tests (p-values):")
names = list(scores.keys())
for i in range(len(names)):
    for j in range(i + 1, len(names)):
        s1, s2 = names[i], names[j]
        _, p = ttest_ind(scores[s1], scores[s2], equal_var=False)
        print(f"{s1} vs {s2}: p = {p:.4g}")

Cohen's d

In [ ]:
print("\nCohen's d (effect size vs Dice-Driven):")
base = "Dice-Driven"
for other in scores:
    if other != base:
        m1, m2 = scores[base].mean(), scores[other].mean()
        s1, s2 = scores[base].std(), scores[other].std()
        pooled_std = ((s1**2 + s2**2) / 2)**0.5
        d = (m1 - m2) / pooled_std
        print(f"{base} vs {other}: d = {d:.2f}")